In [28]:
import MySQLdb
import pandas as pd
import matplotlib as plt
import seaborn as sns
sns.set()

In [3]:
con = MySQLdb.connect(host='relational.fit.cvut.cz', user='guest', passwd='relational', db='imdb_ijs', port=3306 )
cur = con.cursor()

In [4]:
# Atores e filmes ao qual atuaram
atores = cur.execute("select roles.movie_id, concat(actors.first_name, ' ' ,actors.last_name)  from actors inner join roles on roles.actor_id = actors.id order by roles.movie_id;")
atores = pd.DataFrame(cur.fetchall(), columns=['movie_id', 'atores'])
atores = atores.groupby('movie_id').agg(lambda x: x.tolist())

In [5]:
# Diretores e os filmes produzidos
diretores = cur.execute("select movies.id, concat(directors.first_name, ' ' ,directors.last_name)  from movies inner join movies_directors mg on mg.movie_id = movies.id inner join directors on mg.director_id = directors.id;")
diretores = pd.DataFrame(cur.fetchall(), columns=['movie_id','diretores'])
diretores = diretores.groupby('movie_id').agg(lambda x: x.tolist())

In [6]:
# Filmes e os respectivos generos
generos = cur.execute("SELECT movies.id, movies.year, (SELECT  GROUP_CONCAT(genre SEPARATOR ', ') FROM imdb_ijs.movies_genres mvg WHERE mvg.movie_id = movies.id) AS Generos, movies.rank AS Imdb FROM movies where (movies.rank is not null) ORDER BY movies.id;")
generos = pd.DataFrame(cur.fetchall(), columns=['movie_id','ano','genero', 'rating'])
generos = generos.groupby('movie_id').agg(lambda x: x.tolist())

In [7]:
generos.head(5)

,ano,genero,rating
movie_id,,,
2,[1971],"[Comedy, Crime]",[6.4]
6,[1971],"[Comedy, Family]",[5.0]
11,[1939],[Comedy],[6.7]
15,[1999],"[Comedy, Short]",[7.5]
18,[1996],[Comedy],[9.6]


### Formatando Valores para lista

In [8]:
import ast

edit = generos['genero'].astype(str)
edit = edit.map(lambda x: x.replace(',', '\',\'').replace(' ', ''))
edit = pd.DataFrame(edit)
edit = edit.genero.apply(lambda x: list(ast.literal_eval(x)))

all_genders = pd.get_dummies(edit.apply(pd.Series).stack()).sum(level=0)

#### Tratando dados Nulos

In [9]:
generos = generos.dropna()
atores = atores.dropna()
diretores = diretores.dropna()

### Juntando Informações dos filmes

In [10]:
test = pd.merge(diretores, atores, on='movie_id')
test = test.groupby('movie_id').agg(lambda x: x.tolist())
test = pd.merge(test, generos, on='movie_id')
test = test.groupby('movie_id').agg(lambda x: x.tolist())

In [11]:
movies_scores = test.dropna()
aux = movies_scores
aux = aux.dropna()
## Retirando Colunas irrelevantes para recomendação
dt = aux.drop(['diretores', 'atores', 'genero'], axis = 1)

In [12]:
## Retirando colchetes dos tipos de dados
dt.rating = dt.rating.astype(str)
dt.ano = dt.ano.astype(str)
dt.rating = dt.rating.map(lambda x: x.lstrip('[').rstrip(']'))
dt.ano = dt.ano.map(lambda x: x.lstrip('[').rstrip(']'))

dt = pd.merge(dt, all_genders, on='movie_id')


In [13]:
## Dataframe Geral de Informações
dt

,ano,rating,Action,Adult,Adventure,Animation,Comedy,Crime,Documentary,Drama,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Short,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
2,1971,6.4,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1971,5.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,1939,6.7,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1999,7.5,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
18,1996,9.6,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378550,2000,7.0,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
378558,1989,6.9,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
378583,2001,8.7,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
movies_scores = dt
## Separando nota das variáveis independentes
x = movies_scores.drop(['rating'], axis=1)
y = movies_scores.rating

new_x = x.reset_index(drop=True)
new_y = y.reset_index(drop=True)

In [15]:
# Referenciando apenas gêneros ou multi-gêneros para recomendação de filmes
movie_features_df = new_x.drop(["ano"],axis=1)

# Variável auxiliar para mapear o nome dos filmes
movie2_features_df = dt.drop(["ano"],axis=1)
to_map = list(movie2_features_df.index)

In [16]:
movie_features_df.head()

,Action,Adult,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Short,Thriller,War,Western
0,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Mapeando Valores para identificação de Filme

In [17]:
idMovie = {}
MovieName = {}

In [18]:
#print(len(to_map))
movies_name = cur.execute("select movies.id, movies.name from movies")
movies_name = pd.DataFrame(cur.fetchall(), columns=['movie_id','name'])



In [19]:
## Mapeando os nomes
for i in to_map:
    MovieName[i] = str(movies_name.loc[movies_name['movie_id'] == i].name.values)

# Mapeando id´s
rg = range(0,len(to_map))
for i in rg:
    idMovie[i] = to_map[i]

In [20]:
# Juntando o nome dos filmes e infos pelo id
movies_name = pd.merge(movies_name, movie2_features_df, on='movie_id')
movies_name


,movie_id,name,rating,Action,Adult,Adventure,Animation,Comedy,Crime,Documentary,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Short,Thriller,War,Western
0,377575,arrivato mio fratello,5.1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,376875,belles dents,5.5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,376882,bout de souffle,7.8,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,376888,cause d'elle,6.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,376891,"cause, cause d'une femme",7.2,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55785,376518,Zycie jako smiertelna choroba przenoszona drog...,6.7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55786,376522,Zycie rodzinne,6.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55787,376535,Zywot Mateusza,4.8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55788,376540,Zzang,4.4,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### KNN classificador para indicação

In [21]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [22]:
import numpy as np

movie_features_df.shape
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)
query_index =15

distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)


26934


In [23]:
## Lista para alocação de informações(nome e identificador)
ids = []
movie = []
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recomendações para {0}:\n'.format(MovieName[idMovie[movie_features_df.index[query_index]]]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, MovieName[idMovie[movie_features_df.index[indices.flatten()[i]]]], distances.flatten()[i]))
        ids.append(idMovie[movie_features_df.index[indices.flatten()[i]]])
        movie.append(MovieName[idMovie[movie_features_df.index[indices.flatten()[i]]]])

Recomendações para ["'Broadway Melody of 1940'"]:

1: ['Vida sigue igual, La'], with distance of 0.0:
2: ['6.5 Special'], with distance of 0.0:
3: ['Painting the Clouds with Sunshine'], with distance of 0.0:
4: ['Night Magic'], with distance of 0.0:
5: ['Summer Holiday'], with distance of 0.0:


### Dataframe Recomendações 

In [27]:
pred= ids

suggestions = pd.DataFrame(pred)
names=pd.DataFrame(movie)
names=names.reset_index(drop=True)
suggestions['nome']=names
suggestions.columns=['idMovie','Nome']
suggestions


,idMovie,Nome
0,353638,"['Vida sigue igual, La']"
1,2861,['6.5 Special']
2,246749,['Painting the Clouds with Sunshine']
3,231997,['Night Magic']
4,318715,['Summer Holiday']
